In [80]:
# Importing relevant libraris
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import pathlib as pl
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint

In [84]:
def select_class(path):
    if(path == 'Datathon-Dataset\category1_tumor'):
        return 0
    elif(path == 'Datathon-Dataset\category2_tumor'):
        return 1
    elif(path == 'Datathon-Dataset\category3_tumor'):
        return 2
    else:
        return 3
    

In [85]:

# function to create the data set
def create_dataset(path):
    x,y = [],[]
    for subdir in pl.Path(path).iterdir():
        if subdir.is_dir():
            data_dir = pl.Path(subdir)
            file_list = list(data_dir.glob('*.jpg'))
            
            for file in file_list:
                file_path = data_dir.joinpath(file.name)
                img = cv2.imread(str(file_path))
                resized_img = cv2.resize(img,(150,150))
                x.append(resized_img)
                y.append(select_class(str(subdir)))
    return x,y

In [86]:
x,y = create_dataset('Datathon-Dataset')

In [87]:
X = np.array(x)
Y = np.array(y)

In [88]:
# Shuffeling the data set
X,Y = shuffle(X,Y)

In [89]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.1 ,random_state = 100)

In [90]:
efficient_net = EfficientNetB0(weights = 'imagenet',include_top = False,input_shape = (150,150,3))

In [91]:
model = efficient_net.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=efficient_net.input, outputs = model)

In [92]:
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [93]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [94]:
history = model.fit(X_train,Y_train,validation_split=0.1, epochs =12, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])

Epoch 1/12
82/82 [==============================] - ETA: 0s - loss: 0.4882 - accuracy: 0.8146
Epoch 1: val_accuracy improved from -inf to 0.86254, saving model to effnet.h5


C:\Users\Amila Kasun\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


82/82 [==============================] - 242s 3s/step - loss: 0.4882 - accuracy: 0.8146 - val_loss: 0.5287 - val_accuracy: 0.8625 - lr: 0.0010
Epoch 2/12
82/82 [==============================] - ETA: 0s - loss: 0.1665 - accuracy: 0.9483
Epoch 2: val_accuracy did not improve from 0.86254
82/82 [==============================] - 221s 3s/step - loss: 0.1665 - accuracy: 0.9483 - val_loss: 0.5421 - val_accuracy: 0.8591 - lr: 0.0010
Epoch 3/12
82/82 [==============================] - ETA: 0s - loss: 0.1060 - accuracy: 0.9678
Epoch 3: val_accuracy did not improve from 0.86254

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
82/82 [==============================] - 210s 3s/step - loss: 0.1060 - accuracy: 0.9678 - val_loss: 0.8296 - val_accuracy: 0.7973 - lr: 0.0010
Epoch 4/12
82/82 [==============================] - ETA: 0s - loss: 0.0750 - accuracy: 0.9759
Epoch 4: val_accuracy improved from 0.86254 to 0.92096, saving model to effnet.h5
82/82 [=====================

In [121]:
pred = model.predict(X_test)


11/11 [==============================] - 6s 574ms/step


In [97]:
from sklearn.metrics import classification_report,confusion_matrix

In [122]:
pred = np.argmax(pred, axis=1)


In [123]:
pred

array([0, 3, 2, 2, 0, 0, 0, 2, 2, 3, 1, 1, 0, 0, 2, 2, 2, 1, 3, 0, 2, 2,
       2, 1, 3, 1, 0, 2, 3, 1, 0, 2, 2, 3, 3, 3, 1, 0, 1, 1, 2, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0, 3, 1, 0, 0, 1, 0, 2, 0, 1,
       3, 2, 0, 3, 1, 0, 2, 3, 2, 1, 0, 2, 0, 3, 1, 3, 0, 1, 2, 3, 3, 0,
       2, 0, 1, 0, 1, 1, 0, 1, 3, 2, 1, 2, 0, 1, 3, 0, 1, 0, 2, 2, 1, 1,
       2, 3, 2, 0, 2, 1, 3, 2, 1, 1, 3, 1, 1, 0, 2, 0, 1, 0, 2, 1, 0, 2,
       2, 3, 1, 1, 1, 0, 1, 1, 2, 1, 2, 2, 2, 0, 0, 1, 3, 2, 2, 1, 0, 1,
       2, 2, 1, 2, 2, 0, 1, 2, 2, 3, 0, 2, 2, 0, 2, 2, 2, 3, 3, 1, 0, 1,
       3, 1, 1, 0, 2, 1, 3, 2, 1, 1, 3, 0, 2, 0, 2, 3, 2, 2, 2, 1, 2, 2,
       0, 3, 1, 3, 0, 1, 1, 0, 0, 3, 3, 2, 2, 1, 0, 3, 0, 2, 1, 1, 3, 2,
       1, 0, 1, 0, 2, 0, 0, 0, 2, 1, 3, 2, 2, 0, 2, 1, 3, 2, 0, 2, 0, 0,
       0, 1, 0, 3, 2, 2, 2, 1, 0, 0, 0, 0, 1, 0, 2, 0, 3, 1, 1, 1, 0, 0,
       3, 0, 2, 1, 1, 2, 2, 2, 3, 3, 0, 2, 0, 1, 2, 2, 0, 2, 3, 1, 0, 0,
       0, 0, 2, 0, 2, 3, 2, 2, 3, 2, 1, 2, 0, 3, 0,

In [110]:
Y_test

array([0, 3, 2, 2, 0, 0, 0, 2, 2, 3, 1, 1, 0, 0, 2, 2, 2, 1, 3, 0, 2, 2,
       2, 1, 3, 1, 0, 2, 3, 1, 0, 2, 2, 3, 3, 3, 1, 0, 1, 1, 2, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0, 3, 1, 0, 0, 1, 0, 2, 0, 1,
       3, 2, 0, 3, 1, 0, 2, 3, 2, 1, 0, 2, 0, 3, 1, 3, 0, 1, 2, 3, 3, 0,
       2, 0, 1, 0, 1, 1, 0, 1, 3, 2, 1, 2, 0, 1, 3, 0, 1, 0, 2, 2, 1, 1,
       2, 3, 2, 0, 2, 1, 3, 2, 1, 1, 3, 1, 1, 0, 2, 0, 1, 0, 2, 0, 0, 2,
       2, 3, 1, 1, 1, 0, 1, 1, 2, 1, 2, 2, 2, 0, 0, 1, 3, 2, 2, 1, 0, 1,
       2, 2, 1, 2, 2, 0, 1, 2, 2, 3, 0, 2, 2, 0, 2, 2, 2, 3, 3, 1, 0, 1,
       3, 1, 1, 0, 2, 1, 3, 2, 1, 1, 3, 0, 2, 0, 2, 3, 2, 2, 2, 1, 2, 2,
       0, 3, 1, 3, 0, 1, 1, 0, 0, 3, 3, 2, 2, 1, 0, 3, 0, 2, 1, 1, 3, 2,
       1, 0, 1, 0, 2, 0, 0, 0, 2, 1, 3, 2, 2, 0, 2, 1, 3, 2, 0, 2, 0, 0,
       0, 1, 0, 3, 2, 2, 2, 1, 0, 0, 0, 0, 1, 0, 2, 0, 3, 1, 1, 1, 0, 0,
       3, 0, 2, 1, 1, 2, 2, 2, 3, 3, 0, 2, 0, 1, 2, 2, 0, 2, 3, 1, 0, 0,
       0, 0, 2, 0, 2, 3, 2, 2, 3, 2, 1, 2, 0, 3, 0,

In [124]:
print(classification_report(Y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        92
           1       0.98      1.00      0.99        85
           2       1.00      1.00      1.00        95
           3       1.00      1.00      1.00        51

    accuracy                           0.99       323
   macro avg       0.99      0.99      0.99       323
weighted avg       0.99      0.99      0.99       323



In [156]:
model.save('MRI_model.h5')

C:\Users\Amila Kasun\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [130]:
img = cv2.imread('Datathon-Dataset\category1_tumor\gt_img (1).jpg')
img = cv2.resize(img,(150,150))
img

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [157]:
loaded_model = tf.keras.models.load_model("MRI_model.h5")

In [158]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [159]:
image_path = "mt_img (55).jpg"  # Replace with the path to your image
image = load_img(image_path, target_size=(150, 150))  # Resize to match your model's input size
image_array = img_to_array(image)
 # Normalize pixel values (assuming you did this during training)

# Expand dimensions to create a batch of one image
image_batch = np.expand_dims(image_array, axis=0)

In [160]:
prediction = loaded_model.predict(image_batch)

1/1 [==============================] - 6s 6s/step


In [161]:
np.argmax(prediction)

1